In [1]:
import os
import pandas as pd
from datetime import datetime


In [2]:
os.chdir("C:\\Users\\ywexl\\OneDrive\\Desktop\\DS&A")

In [3]:
def Income(user_id):
    income_file_path = f"Income_{user_id}.csv"

    if os.path.exists(income_file_path):
        income_df = pd.read_csv(income_file_path)
    else:
        income_df = pd.DataFrame(columns=["timestamp", "user_id", "income", "source"])

    def Add_Income():
        income = float(input("Enter your income: "))
        income_source = input("Enter your income source: ")
        income_entry = pd.DataFrame({
            "timestamp": [datetime.now().strftime('%Y-%m-%d %H:%M:%S')],
            "user_id": [user_id],
            "income": [income],
            "source": [income_source]
        })
        nonlocal income_df
        income_df = pd.concat([income_df, income_entry], ignore_index=True)
        income_df.to_csv(income_file_path, index=False)

        update_savings(user_id, income)
        print(f"Income of {income} added from source '{income_source}'.")

    while True:
        Add_Income()
        add_another = input("Do you want to add another income? (yes/no): ").strip().lower()
        if add_another != "yes":
            break


def Expenses(user_id):
    expenses_file_path = f"Expenses_{user_id}.csv"

    if os.path.exists(expenses_file_path):
        expenses_df = pd.read_csv(expenses_file_path)
    else:
        expenses_df = pd.DataFrame(columns=["timestamp", "user_id", "expense", "category"])

    def Add_Expense():
        expense = float(input("Enter your expense: "))
        expense_category = input("Enter your expense category: ")
        expense_entry = pd.DataFrame({
            "timestamp": [datetime.now().strftime('%Y-%m-%d %H:%M:%S')],
            "user_id": [user_id],
            "expense": [expense],
            "category": [expense_category]
        })
        nonlocal expenses_df
        expenses_df = pd.concat([expenses_df, expense_entry], ignore_index=True)
        expenses_df.to_csv(expenses_file_path, index=False)

        update_savings(user_id, -expense)
        print(f"Expense of {expense} added under category '{expense_category}'.")

    while True:
        Add_Expense()
        add_another = input("Do you want to add another expense? (yes/no): ").strip().lower()
        if add_another != "yes":
            break


def update_savings(user_id, net_change, retirement_goal=None, user_age=None):
    path = f"Savings_{user_id}.csv"

    required_cols = ["timestamp", "user_id", "retirement_goal", "user_age", "current_savings"]
    if not os.path.exists(path):
        print(f"Initializing savings file for user {user_id}.")
        savings_df = pd.DataFrame(columns=required_cols)
        current_savings = 0
        savings_df.to_csv(path, index=False)
    else:
        savings_df = pd.read_csv(path)

        if savings_df.empty or user_id not in savings_df["user_id"].values:
            print(f"The savings file for user {user_id} is empty or missing required columns.")
            current_savings = 0
        else:
            if user_id in savings_df["user_id"].values:
                current_savings = savings_df[savings_df["user_id"] == user_id]["current_savings"].iloc[-1]
                if retirement_goal is None:
                    retirement_goal = savings_df[savings_df["user_id"] == user_id]["retirement_goal"].iloc[-1]
                if user_age is None:
                    user_age = savings_df[savings_df["user_id"] == user_id]["user_age"].iloc[-1]
            else:
                current_savings = 0

# Make sure columns exist
    for col in required_cols:
        if col not in savings_df.columns:
            savings_df[col] = None

    updated_savings = current_savings + net_change

    new_entry = {
        "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "user_id": user_id,
        "retirement_goal": retirement_goal,
        "user_age": user_age,
        "current_savings": updated_savings
    }
    savings_df = pd.concat([savings_df, pd.DataFrame([new_entry])], ignore_index=True)

    savings_df.to_csv(path, index=False)

    print(f"Savings updated successfully! Current savings: {updated_savings}")

def PaceCalculator(user_id, retirement_goal, user_age):
    savings_file_path = f"Savings_{user_id}.csv"

    # Load latest savings
    if os.path.exists(savings_file_path):
        savings_df = pd.read_csv(savings_file_path)
        if not savings_df.empty and "current_savings" in savings_df.columns:
            current_savings = savings_df[savings_df["user_id"] == user_id]["current_savings"].iloc[-1]
        else:
            current_savings = 0
    else:
        print(f"No savings file found for user {user_id}. Initializing with default values.")
        current_savings = 0

    # years to retirement
    retirement_age = 65  #Default
    years_to_retirement = retirement_age - user_age

    if years_to_retirement <= 0:
        print("You have already reached or passed the retirement age!")
        return

    # Monthly savings needed 
    amount_needed = retirement_goal - current_savings
    if amount_needed <= 0:
        print("Congratulations! You have already reached your retirement goal!")
        return

    monthly_contribution = amount_needed / (years_to_retirement * 12)

    print(f"To reach your retirement goal of {retirement_goal}, you need to save:")
    print(f"- Total amount needed: {amount_needed:.2f}")
    print(f"- Monthly savings required: {monthly_contribution:.2f}")
    print(f"- Years to retirement: {years_to_retirement}")

def HomePage():
    def display_Dashboard():
        global user_id, retirement_goal, user_age
        user_id = input("Enter your user ID: ")
        if not user_id.isdigit():
            print("Invalid user ID. Please enter a numeric value.")
            return
        user_id = int(user_id)

        savings_file_path = f"Savings_{user_id}.csv"

        
        if os.path.exists(savings_file_path):
            savings_df = pd.read_csv(savings_file_path)

# Check for retirement goal and age
            if not savings_df.empty and "retirement_goal" in savings_df.columns and "user_age" in savings_df.columns:
                user_data = savings_df[savings_df["user_id"] == user_id]
                if not user_data.empty:
                    retirement_goal = user_data["retirement_goal"].iloc[-1]
                    user_age = user_data["user_age"].iloc[-1]

                    if pd.isna(retirement_goal) or pd.isna(user_age):
                        print("Some information is missing. Please provide the following details:")
                        retirement_goal = float(input("Enter your retirement goal: "))
                        user_age = int(input("Enter your age: "))
                        update_savings(user_id, 0, retirement_goal, user_age)
                else:
                    print("User ID not found. Please provide the following details:")
                    retirement_goal = float(input("Enter your retirement goal: "))
                    user_age = int(input("Enter your age: "))
                    update_savings(user_id, 0, retirement_goal, user_age)
            else:
                print("Savings file is missing required information. Please provide the following details:")
                retirement_goal = float(input("Enter your retirement goal: "))
                user_age = int(input("Enter your age: "))
                update_savings(user_id, 0, retirement_goal, user_age)
        else:
            print(f"Initializing savings file for user {user_id}.")
            retirement_goal = float(input("Enter your retirement goal: "))
            user_age = int(input("Enter your age: "))
            savings_df = pd.DataFrame([{
                "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "user_id": user_id,
                "retirement_goal": retirement_goal,
                "user_age": user_age,
                "current_savings": 0
            }])
            savings_df.to_csv(savings_file_path, index=False)

        while True:
            choice = input(
                "Enter your choice! \n"
                "1: Income \n"
                "2: Expenses \n"
                "3: View Retirement Goal Progress \n"
                "4: Exit \n"
                "Your choice: "
            )
            if choice == "1":
                Income(user_id)
            elif choice == "2":
                Expenses(user_id)
            elif choice == "3":
                PaceCalculator(user_id, retirement_goal, user_age)
            elif choice == "4":
                print("Exiting the dashboard. Goodbye!")
                break
            else:
                print("Invalid choice, please try again.")

    display_Dashboard()

In [4]:
HomePage()

Initializing savings file for user 3.


C:\Users\ywexl\AppData\Local\Temp\ipykernel_13140\687675656.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  income_df = pd.concat([income_df, income_entry], ignore_index=True)


Savings updated successfully! Current savings: 100000.0
Income of 100000.0 added from source '$$$'.


C:\Users\ywexl\AppData\Local\Temp\ipykernel_13140\687675656.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  expenses_df = pd.concat([expenses_df, expense_entry], ignore_index=True)


Savings updated successfully! Current savings: -90000.0
Expense of 190000.0 added under category 'travel'.
Exiting the dashboard. Goodbye!
